# <img src="logo.png" alt="Logo" width="70" style="vertical-align: middle; margin-right: 10px;"> Bharat Herald: Data Preparation and Cleaning (2019–2024)

This notebook is part of the **Codebasics Resume Project Challenge**.  
The business case: Bharat Herald, a 70-year-old newspaper brand, is struggling with declining print circulation, advertiser confidence, and a failed digital pilot.  
The mandate is to **quantify what went wrong, identify recovery opportunities, and recommend a phased roadmap for digital transformation**

---

## Objective of this Notebook
The purpose of this notebook is to prepare clean datasets that will be used for:
- **Exploring, understanding and validating data (this notebook)**
- **Visualization and storytelling in Power BI (main analysis + dashboard)**
- **Answering primary and secondary business questions**, such as:
  - Print circulation trends and waste analysis
  - Ad revenue evolution across categories and cities
  - Digital readiness vs. performance
  - ROI analysis and digital relaunch prioritization (the answers are available in the video presentation)

---

## Cleaning Summary
- All raw fact and dimension tables were cleaned, normalized, and standardized.
- Final outputs were exported as Excel files for use in Power BI:
  1. `fact_print_sales_cleaned.xlsx`
  2. `fact_digital_pilot_cleaned.xlsx`
  3. `fact_city_readiness_cleaned.xlsx`
  4. `fact_ad_revenue_cleaned.xlsx`
  5. `dim_city_cleaned.xlsx`
  6. `dim_ad_category_cleaned.xlsx`

- In addition, supporting dimension tables were created and exported by coding within this notebook to make data modeling in Power BI smoother:
  - `dim_date.xlsx`
  - `dim_quarter.xlsx`
  - `dim_year.xlsx`
  - `dim_edition.xlsx`

These cleaned and structured files form the backbone of the analysis model in Power BI.

---

## Why This Approach?

Two common questions may come up while reviewing this notebook:

**1. Why not visualize directly in Python?**  
- Python was used primarily for data cleaning, transformation, and validation because it offers flexibility in handling inconsistent formats (e.g., messy quarter values, missing fields).  
- Power BI was chosen for visualization because it enables interactive dashboards, filters, and drill-downs that stakeholders can explore on their own.  
- In practice, both tools complement each other: Python ensures accuracy, and Power BI ensures accessibility.

**2. Why not clean the data directly in Power BI?**  
- Cleaning in Python ensures the process is reproducible and transparent. Anyone can rerun this notebook to regenerate the same cleaned datasets.  
- Power BI has strong modeling capabilities but limited flexibility for advanced cleaning steps. Python allowed us to standardize formats, validate joins, and apply consistent business rules before loading into Power BI.  
- This separation mirrors real-world analytics workflows:  
  - **Python/SQL** → data preparation  
  - **BI tool (Power BI)** → storytelling and decision-making

---


In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#loading datasets
fact_print_sales = pd.read_excel("fact_print_sales.xlsx")
fact_digital_pilot = pd.read_csv("fact_digital_pilot.csv")
fact_city_readiness = pd.read_csv("fact_city_readiness.csv")
fact_ad_revenue = pd.read_csv("fact_ad_revenue.csv")
dim_city = pd.read_excel('dim_city.xlsx')
dim_ad_category = pd.read_excel('dim_ad_category.xlsx')

In [3]:
#checking if loaded successfully
print("Fact Print Sales:", fact_print_sales.shape)
print("Fact Digital Pilot:", fact_digital_pilot.shape)
print("Fact City Readiness:", fact_city_readiness.shape)
print("Fact Ad Revenue:", fact_ad_revenue.shape)
print("Dim City:", dim_city.shape)
print("Dim Ad Category:", dim_ad_category.shape)

Fact Print Sales: (720, 8)
Fact Digital Pilot: (48, 11)
Fact City Readiness: (240, 6)
Fact Ad Revenue: (720, 6)
Dim City: (10, 4)
Dim Ad Category: (4, 4)


### Data Exploration and Cleaning

#### Dataset 1: fact_print_sales

In [4]:
fact_print_sales.head()

,edition_ID,City_ID,Language,State,Month,Copies Sold,copies_returned,Net_Circulation
0,ED1005,C005,Hindi,Rajasthan,2023-05-01 00:00:00,404389,13510,390879
1,ED1005,C005,Hindi,Rajasthan,2019-03-01 00:00:00,492943,25024,467919
2,ED1001,C001,hindi,Uttar pradesh,2023-07-01 00:00:00,168893,12285,156608
3,ED1003,C003,Hindi,Madhya_Pradesh,2023-07-01 00:00:00,216540,10117,206423
4,ED1007,C007,Hindi,Jharkhand,2020-10-01 00:00:00,234563,13048,221515


In [5]:
fact_print_sales.describe()

,copies_returned,Net_Circulation
count,720.000000,720.000000
mean,16430.815278,287950.551389
std,6524.459623,83191.491771
min,4991.000000,139098.000000
25%,11379.500000,219982.750000
50%,15151.000000,279778.000000
75%,20902.250000,351736.750000
max,38021.000000,487255.000000


In [6]:
fact_print_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   edition_ID       720 non-null    object
 1   City_ID          720 non-null    object
 2   Language         720 non-null    object
 3   State            720 non-null    object
 4   Month            720 non-null    object
 5   Copies Sold      720 non-null    object
 6   copies_returned  720 non-null    int64 
 7   Net_Circulation  720 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 45.1+ KB


In [7]:
#converting column headers to single format camel_case
fact_print_sales.rename(columns={
    'edition_ID': 'edition_id',
    'City_ID': 'city_id',
    'Language': 'language',
    'State': 'state',
    'Month': 'month',
    'Copies Sold': 'copies_sold',
    'copies_returned': 'copies_returned',
    'Net_Circulation': 'net_circulation'
}, inplace=True)

fact_print_sales.head()

,edition_id,city_id,language,state,month,copies_sold,copies_returned,net_circulation
0,ED1005,C005,Hindi,Rajasthan,2023-05-01 00:00:00,404389,13510,390879
1,ED1005,C005,Hindi,Rajasthan,2019-03-01 00:00:00,492943,25024,467919
2,ED1001,C001,hindi,Uttar pradesh,2023-07-01 00:00:00,168893,12285,156608
3,ED1003,C003,Hindi,Madhya_Pradesh,2023-07-01 00:00:00,216540,10117,206423
4,ED1007,C007,Hindi,Jharkhand,2020-10-01 00:00:00,234563,13048,221515


In [8]:
# checking why copies_sold column is an object instead of int64 dtype
fact_print_sales['copies_sold'].apply(lambda x: str(x).strip().isdigit()).value_counts()

copies_sold
True     654
False     66
Name: count, dtype: int64

In [9]:
# Show all non-numeric values in copies_sold
fact_print_sales[~fact_print_sales['copies_sold'].apply(lambda x: str(x).strip().isdigit())][['copies_sold']].value_counts()

copies_sold
â‚¹152122      1
â‚¹170434      1
â‚¹172787      1
â‚¹182586      1
â‚¹190618      1
              ..
â‚¹460138      1
â‚¹472940      1
â‚¹480926      1
â‚¹488725      1
â‚¹492394      1
Name: count, Length: 66, dtype: int64

In [10]:
#The prefix â‚¹ is a garbled Unicode version of the Indian Rupee symbol (₹) — likely due to wrong encoding during export.
#Cleaning these non-numeric columns

fact_print_sales['copies_sold'] = (
    fact_print_sales['copies_sold']
    .astype(str)
    .str.replace('â‚¹', '', regex=False)
    .str.strip()
    .astype(int)
)

In [11]:
fact_print_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   edition_id       720 non-null    object
 1   city_id          720 non-null    object
 2   language         720 non-null    object
 3   state            720 non-null    object
 4   month            720 non-null    object
 5   copies_sold      720 non-null    int64 
 6   copies_returned  720 non-null    int64 
 7   net_circulation  720 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 45.1+ KB


In [12]:
# converting month column datatype from object to datetime
fact_print_sales['month'] = pd.to_datetime(fact_print_sales['month'])
fact_print_sales.head()

,edition_id,city_id,language,state,month,copies_sold,copies_returned,net_circulation
0,ED1005,C005,Hindi,Rajasthan,2023-05-01,404389,13510,390879
1,ED1005,C005,Hindi,Rajasthan,2019-03-01,492943,25024,467919
2,ED1001,C001,hindi,Uttar pradesh,2023-07-01,168893,12285,156608
3,ED1003,C003,Hindi,Madhya_Pradesh,2023-07-01,216540,10117,206423
4,ED1007,C007,Hindi,Jharkhand,2020-10-01,234563,13048,221515


In [13]:
fact_print_sales['language'].unique()

array(['Hindi', 'hindi', 'english', 'ENGLISH'], dtype=object)

In [14]:
#Data Cleaning and Standarization

fact_print_sales['language'] = fact_print_sales['language'].replace({
    'hindi': 'Hindi',
    'english': 'English',
    'ENGLISH': 'English'
})
fact_print_sales['language'].unique()

array(['Hindi', 'English'], dtype=object)

In [15]:
fact_print_sales['state'].unique()

array(['Rajasthan', 'Uttar pradesh', 'Madhya_Pradesh', 'Jharkhand',
       'maharashtra', 'Uttar-Pradesh', 'Delhi', 'gujarat', 'bihar',
       'Uttar Pradesh'], dtype=object)

In [16]:
fact_print_sales['state'] = fact_print_sales['state'].replace({
    'Uttar pradesh': 'Uttar Pradesh',
    'Madhya_Pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'Uttar-Pradesh': 'Uttar Pradesh',
    'gujarat': 'Gujarat',
    'bihar': 'Bihar'
})

fact_print_sales['state'].unique()

array(['Rajasthan', 'Uttar Pradesh', 'Madhya Pradesh', 'Jharkhand',
       'Maharashtra', 'Delhi', 'Gujarat', 'Bihar'], dtype=object)

In [17]:
fact_print_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   edition_id       720 non-null    object        
 1   city_id          720 non-null    object        
 2   language         720 non-null    object        
 3   state            720 non-null    object        
 4   month            720 non-null    datetime64[ns]
 5   copies_sold      720 non-null    int64         
 6   copies_returned  720 non-null    int64         
 7   net_circulation  720 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 45.1+ KB


In [18]:
fact_print_sales.describe()

,month,copies_sold,copies_returned,net_circulation
count,720,720.000000,720.000000,720.000000
mean,2021-12-15 22:40:00,304381.366667,16430.815278,287950.551389
min,2019-01-01 00:00:00,146927.000000,4991.000000,139098.000000
25%,2020-06-23 12:00:00,231676.500000,11379.500000,219982.750000
50%,2021-12-16 12:00:00,297188.500000,15151.000000,279778.000000
75%,2023-06-08 12:00:00,372676.000000,20902.250000,351736.750000
max,2024-12-01 00:00:00,511248.000000,38021.000000,487255.000000
std,NaN,87719.391755,6524.459623,83191.491771


In [19]:
#checking for outliers 
mean_cs = fact_print_sales['copies_sold'].mean()
std_cs = fact_print_sales['copies_sold'].std()
max_cs = fact_print_sales['copies_sold'].max()
min_cs = fact_print_sales['copies_sold'].min()

mean_cr = fact_print_sales['copies_returned'].mean()
std_cr = fact_print_sales['copies_returned'].std()
max_cr = fact_print_sales['copies_returned'].max()
min_cr = fact_print_sales['copies_returned'].min()

mean_nc = fact_print_sales['net_circulation'].mean()
std_nc = fact_print_sales['net_circulation'].std()
max_nc = fact_print_sales['net_circulation'].max()
min_nc = fact_print_sales['net_circulation'].min()

lower_bound_cs = mean_cs - 3 * std_cs
upper_bound_cs = mean_cs + 3 * std_cs

lower_bound_cr = mean_cr - 3 * std_cr
upper_bound_cr = mean_cr + 3 * std_cr

lower_bound_nc = mean_nc - 3 * std_nc
upper_bound_nc = mean_nc + 3 * std_nc

print(f"Copies Sold: (Lower:{int(lower_bound_cs)}, Min: {int(min_cs)}, Max: {int(max_cs)}, Upper: {int(upper_bound_cs)})")
print(f"Copies Returned: (Lower:{int(lower_bound_cr)}, Min: {int(min_cr)}, Max: {int(max_cr)}, Upper: {int(upper_bound_cr)}) ")
print(f"Net Circulation: (Lower:{int(lower_bound_nc)}, Min: {int(min_nc)}, Max: {int(max_nc)}, Upper: {int(upper_bound_nc)}) ")

Copies Sold: (Lower:41223, Min: 146927, Max: 511248, Upper: 567539)
Copies Returned: (Lower:-3142, Min: 4991, Max: 38021, Upper: 36004) 
Net Circulation: (Lower:38376, Min: 139098, Max: 487255, Upper: 537525) 


##### We notice max amount of copies returned is a little bit higher than the upper limit, but not high enough to be treated as outlier.
##### All other values fall within the upper and lower limits. Hence no outliers detected.

#### Dataset 2: fact_digital_pilot

In [20]:
fact_digital_pilot.head()

,Unnamed: 0,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
0,0,PDF WhatsApp Push,2021-01,A001,236570,66060,23509,16319,52.55,"Mixed feedback: some usability concerns, but h...",C001
1,1,PDF WhatsApp Push,2021-02,A001,156865,99122,19472,17017,82.53,"Mixed feedback: some usability concerns, but h...",C002
2,2,PDF WhatsApp Push,2021-03,A001,242728,46087,8471,2891,68.06,"Mixed feedback: some usability concerns, but h...",C003
3,3,PDF WhatsApp Push,2021-04,A001,147695,78868,46796,15640,66.17,"Mixed feedback: some usability concerns, but h...",C004
4,4,PDF WhatsApp Push,2021-05,A001,325906,135644,16805,3231,76.90,The site takes too long to load on average pho...,C005


In [21]:
fact_digital_pilot.describe()

,Unnamed: 0,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate
count,48.00,48.000000,48.000000,48.000000,48.000000,48.000000
mean,23.50,184276.687500,69980.083333,25671.625000,14225.770833,65.767292
std,14.00,69989.993082,27382.612710,12833.895367,8903.425273,14.452675
min,0.00,82023.000000,31651.000000,6619.000000,2880.000000,41.360000
25%,11.75,138414.750000,50261.750000,13246.500000,6019.500000,52.397500
50%,23.50,184043.500000,65055.500000,23751.500000,13727.000000,66.070000
75%,35.25,226451.000000,86073.250000,34309.750000,21454.000000,78.030000
max,47.00,415324.000000,167913.000000,48619.000000,30763.000000,89.250000


In [22]:
fact_digital_pilot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          48 non-null     int64  
 1   platform                            48 non-null     object 
 2   launch_month                        48 non-null     object 
 3   ad_category_id                      48 non-null     object 
 4   dev_cost                            48 non-null     int64  
 5   marketing_cost                      48 non-null     int64  
 6   users_reached                       48 non-null     int64  
 7   downloads_or_accesses               48 non-null     int64  
 8   avg_bounce_rate                     48 non-null     float64
 9   cumulative_feedback_from_customers  47 non-null     object 
 10  city_id                             48 non-null     object 
dtypes: float64(1), int64(5), object(5)
memory usage

In [23]:
# converting launch month column datatype from object to datetime
fact_digital_pilot['launch_month']=pd.to_datetime(fact_digital_pilot['launch_month'], format='%Y-%m')
fact_digital_pilot.head()

,Unnamed: 0,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
0,0,PDF WhatsApp Push,2021-01-01,A001,236570,66060,23509,16319,52.55,"Mixed feedback: some usability concerns, but h...",C001
1,1,PDF WhatsApp Push,2021-02-01,A001,156865,99122,19472,17017,82.53,"Mixed feedback: some usability concerns, but h...",C002
2,2,PDF WhatsApp Push,2021-03-01,A001,242728,46087,8471,2891,68.06,"Mixed feedback: some usability concerns, but h...",C003
3,3,PDF WhatsApp Push,2021-04-01,A001,147695,78868,46796,15640,66.17,"Mixed feedback: some usability concerns, but h...",C004
4,4,PDF WhatsApp Push,2021-05-01,A001,325906,135644,16805,3231,76.90,The site takes too long to load on average pho...,C005


In [24]:
#dropping unnamed column
fact_digital_pilot.drop(columns=['Unnamed: 0'], inplace=True)
fact_digital_pilot.head()

,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
0,PDF WhatsApp Push,2021-01-01,A001,236570,66060,23509,16319,52.55,"Mixed feedback: some usability concerns, but h...",C001
1,PDF WhatsApp Push,2021-02-01,A001,156865,99122,19472,17017,82.53,"Mixed feedback: some usability concerns, but h...",C002
2,PDF WhatsApp Push,2021-03-01,A001,242728,46087,8471,2891,68.06,"Mixed feedback: some usability concerns, but h...",C003
3,PDF WhatsApp Push,2021-04-01,A001,147695,78868,46796,15640,66.17,"Mixed feedback: some usability concerns, but h...",C004
4,PDF WhatsApp Push,2021-05-01,A001,325906,135644,16805,3231,76.90,The site takes too long to load on average pho...,C005


In [25]:
fact_digital_pilot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   platform                            48 non-null     object        
 1   launch_month                        48 non-null     datetime64[ns]
 2   ad_category_id                      48 non-null     object        
 3   dev_cost                            48 non-null     int64         
 4   marketing_cost                      48 non-null     int64         
 5   users_reached                       48 non-null     int64         
 6   downloads_or_accesses               48 non-null     int64         
 7   avg_bounce_rate                     48 non-null     float64       
 8   cumulative_feedback_from_customers  47 non-null     object        
 9   city_id                             48 non-null     object        
dtypes: datetime64[ns](1), float6

In [26]:
# dividing percentage values by 100 for consistent scale across datasets
fact_digital_pilot['avg_bounce_rate'] = fact_digital_pilot['avg_bounce_rate'].div(100)

In [27]:
fact_digital_pilot.head()

,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
0,PDF WhatsApp Push,2021-01-01,A001,236570,66060,23509,16319,0.5255,"Mixed feedback: some usability concerns, but h...",C001
1,PDF WhatsApp Push,2021-02-01,A001,156865,99122,19472,17017,0.8253,"Mixed feedback: some usability concerns, but h...",C002
2,PDF WhatsApp Push,2021-03-01,A001,242728,46087,8471,2891,0.6806,"Mixed feedback: some usability concerns, but h...",C003
3,PDF WhatsApp Push,2021-04-01,A001,147695,78868,46796,15640,0.6617,"Mixed feedback: some usability concerns, but h...",C004
4,PDF WhatsApp Push,2021-05-01,A001,325906,135644,16805,3231,0.7690,The site takes too long to load on average pho...,C005


In [28]:
fact_digital_pilot['platform'].unique()

array(['PDF WhatsApp Push', 'E-paper Mobile Web', 'Mobile App Beta',
       'Responsive Web Version'], dtype=object)

In [29]:
fact_digital_pilot['cumulative_feedback_from_customers'].unique()

array(['Mixed feedback: some usability concerns, but high interest in WhatsApp format.',
       'The site takes too long to load on average phones.\nPeople didn’t wait around.',
       'Many said font was too tiny to read.\nZooming didn’t help on small screens.',
       nan], dtype=object)

In [30]:
fact_digital_pilot.describe()

,launch_month,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate
count,48,48.000000,48.000000,48.000000,48.000000,48.000000
mean,2021-06-16 12:00:00,184276.687500,69980.083333,25671.625000,14225.770833,0.657673
min,2021-01-01 00:00:00,82023.000000,31651.000000,6619.000000,2880.000000,0.413600
25%,2021-03-24 06:00:00,138414.750000,50261.750000,13246.500000,6019.500000,0.523975
50%,2021-06-16 00:00:00,184043.500000,65055.500000,23751.500000,13727.000000,0.660700
75%,2021-09-08 12:00:00,226451.000000,86073.250000,34309.750000,21454.000000,0.780300
max,2021-12-01 00:00:00,415324.000000,167913.000000,48619.000000,30763.000000,0.892500
std,NaN,69989.993082,27382.612710,12833.895367,8903.425273,0.144527


In [31]:
#checking for outliers 
mean_dc = fact_digital_pilot['dev_cost'].mean()
std_dc = fact_digital_pilot['dev_cost'].std()
max_dc = fact_digital_pilot['dev_cost'].max()
min_dc = fact_digital_pilot['dev_cost'].min()

mean_mc = fact_digital_pilot['marketing_cost'].mean()
std_mc = fact_digital_pilot['marketing_cost'].std()
max_mc = fact_digital_pilot['marketing_cost'].max()
min_mc = fact_digital_pilot['marketing_cost'].min()

mean_ur = fact_digital_pilot['users_reached'].mean()
std_ur = fact_digital_pilot['users_reached'].std()
max_ur = fact_digital_pilot['users_reached'].max()
min_ur = fact_digital_pilot['users_reached'].min()

mean_da = fact_digital_pilot['downloads_or_accesses'].mean()
std_da = fact_digital_pilot['downloads_or_accesses'].std()
max_da = fact_digital_pilot['downloads_or_accesses'].max()
min_da = fact_digital_pilot['downloads_or_accesses'].min()

mean_abr = fact_digital_pilot['avg_bounce_rate'].mean()
std_abr = fact_digital_pilot['avg_bounce_rate'].std()
max_abr = fact_digital_pilot['avg_bounce_rate'].max()
min_abr = fact_digital_pilot['avg_bounce_rate'].min()

lower_bound_dc = mean_dc - 3 * std_dc
upper_bound_dc = mean_dc + 3 * std_dc

lower_bound_mc = mean_mc - 3 * std_mc
upper_bound_mc = mean_mc + 3 * std_mc

lower_bound_ur = mean_ur - 3 * std_ur
upper_bound_ur = mean_ur + 3 * std_ur

lower_bound_da = mean_da - 3 * std_da
upper_bound_da = mean_da + 3 * std_da

lower_bound_abr = mean_abr - 3 * std_abr
upper_bound_abr = mean_abr + 3 * std_abr

print(f"Dev Cost: (Lower:{int(lower_bound_dc)}, Min: {int(min_dc)}, Max: {int(max_dc)}, Upper: {int(upper_bound_dc)})" )
print(f"Marketing Cost: (Lower:{int(lower_bound_mc)}, Min: {int(min_mc)}, Max: {int(max_mc)}, Upper: {int(upper_bound_mc)})")
print(f"Users Reached: (Lower:{int(lower_bound_ur)}, Min: {int(min_ur)}, Max: {int(max_ur)}, Upper: {int(upper_bound_ur)})")
print(f"Downloads or Accesses: (Lower:{int(lower_bound_da)}, Min: {int(min_da)}, Max: {int(max_da)}, Upper: {int(upper_bound_da)})")
print(f"Avg Bounce Rate: (Lower:{round(lower_bound_abr,2)}, Min: {round(min_abr,2)}, Max: {round(max_abr,2)}, Upper: {round(upper_bound_abr,2)})")

Dev Cost: (Lower:-25693, Min: 82023, Max: 415324, Upper: 394246)
Marketing Cost: (Lower:-12167, Min: 31651, Max: 167913, Upper: 152127)
Users Reached: (Lower:-12830, Min: 6619, Max: 48619, Upper: 64173)
Downloads or Accesses: (Lower:-12484, Min: 2880, Max: 30763, Upper: 40936)
Avg Bounce Rate: (Lower:0.22, Min: 0.41, Max: 0.89, Upper: 1.09)


##### Maximum amounts in Development Cost and Marketing cost are slightly higher than the upper limit, but not high enough to be treated as outliers.
##### All other values fall within the upper and lower limits. Hence no outliers detected.

#### Dataset 3: fact_city_readiness

In [32]:
fact_city_readiness.head()

,Unnamed: 0,city_id,quarter,literacy_rate,smartphone_penetration,internet_penetration
0,0,C001,2019-Q1,89.16,75.76,56.53
1,1,C001,2019-Q2,88.76,76.45,55.97
2,2,C001,2019-Q3,88.83,75.32,56.52
3,3,C001,2019-Q4,89.25,75.83,56.94
4,4,C001,2020-Q1,89.13,75.03,56.45


In [33]:
fact_city_readiness.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              240 non-null    int64  
 1   city_id                 240 non-null    object 
 2   quarter                 240 non-null    object 
 3   literacy_rate           240 non-null    float64
 4   smartphone_penetration  240 non-null    float64
 5   internet_penetration    240 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 11.4+ KB


In [34]:
#dropping unnamed column
fact_city_readiness.drop(columns=['Unnamed: 0'], inplace=True)
fact_city_readiness.head()

,city_id,quarter,literacy_rate,smartphone_penetration,internet_penetration
0,C001,2019-Q1,89.16,75.76,56.53
1,C001,2019-Q2,88.76,76.45,55.97
2,C001,2019-Q3,88.83,75.32,56.52
3,C001,2019-Q4,89.25,75.83,56.94
4,C001,2020-Q1,89.13,75.03,56.45


In [35]:
fact_city_readiness['quarter'].unique()

array(['2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1', '2020-Q2',
       '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4',
       '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1', '2023-Q2',
       '2023-Q3', '2023-Q4', '2024-Q1', '2024-Q2', '2024-Q3', '2024-Q4'],
      dtype=object)

In [36]:
#splitting year and quarters

fact_city_readiness['year'] = fact_city_readiness['quarter'].str[:4]
fact_city_readiness['quarter_only'] = fact_city_readiness['quarter'].str[-2:]
fact_city_readiness.head()

,city_id,quarter,literacy_rate,smartphone_penetration,internet_penetration,year,quarter_only
0,C001,2019-Q1,89.16,75.76,56.53,2019,Q1
1,C001,2019-Q2,88.76,76.45,55.97,2019,Q2
2,C001,2019-Q3,88.83,75.32,56.52,2019,Q3
3,C001,2019-Q4,89.25,75.83,56.94,2019,Q4
4,C001,2020-Q1,89.13,75.03,56.45,2020,Q1


In [37]:
fact_city_readiness.drop(columns=['quarter'], inplace = True)
fact_city_readiness.head()

,city_id,literacy_rate,smartphone_penetration,internet_penetration,year,quarter_only
0,C001,89.16,75.76,56.53,2019,Q1
1,C001,88.76,76.45,55.97,2019,Q2
2,C001,88.83,75.32,56.52,2019,Q3
3,C001,89.25,75.83,56.94,2019,Q4
4,C001,89.13,75.03,56.45,2020,Q1


In [38]:
fact_city_readiness.rename(columns={'quarter_only': 'quarter'}, inplace=True)
fact_city_readiness.head()

,city_id,literacy_rate,smartphone_penetration,internet_penetration,year,quarter
0,C001,89.16,75.76,56.53,2019,Q1
1,C001,88.76,76.45,55.97,2019,Q2
2,C001,88.83,75.32,56.52,2019,Q3
3,C001,89.25,75.83,56.94,2019,Q4
4,C001,89.13,75.03,56.45,2020,Q1


In [39]:
# dividing percentage values by 100 for consistent scale across datasets
pct_cols = ["literacy_rate", "smartphone_penetration", "internet_penetration"]
fact_city_readiness[pct_cols] = fact_city_readiness[pct_cols].div(100)
fact_city_readiness.head()

,city_id,literacy_rate,smartphone_penetration,internet_penetration,year,quarter
0,C001,0.8916,0.7576,0.5653,2019,Q1
1,C001,0.8876,0.7645,0.5597,2019,Q2
2,C001,0.8883,0.7532,0.5652,2019,Q3
3,C001,0.8925,0.7583,0.5694,2019,Q4
4,C001,0.8913,0.7503,0.5645,2020,Q1


In [40]:
# Calculating City Readiness Score 
fact_city_readiness['city_readiness_score'] = (fact_city_readiness['literacy_rate']+fact_city_readiness['smartphone_penetration']+fact_city_readiness['internet_penetration'])/3


In [41]:
fact_city_readiness.describe()

,literacy_rate,smartphone_penetration,internet_penetration,city_readiness_score
count,240.000000,240.000000,240.000000,240.000000
mean,0.768356,0.682914,0.610777,0.687349
std,0.070387,0.105426,0.190297,0.069715
min,0.661100,0.454400,0.100000,0.541900
25%,0.708050,0.676250,0.564475,0.680633
50%,0.754150,0.705400,0.671250,0.714100
75%,0.827825,0.763575,0.743300,0.735808
max,0.894300,0.804100,0.771000,0.757767


In [42]:
#checking for outliers 
mean_lr = fact_city_readiness['literacy_rate'].mean()
std_lr = fact_city_readiness['literacy_rate'].std()
max_lr = fact_city_readiness['literacy_rate'].max()
min_lr = fact_city_readiness['literacy_rate'].min()

mean_sp = fact_city_readiness['smartphone_penetration'].mean()
std_sp = fact_city_readiness['smartphone_penetration'].std()
max_sp = fact_city_readiness['smartphone_penetration'].max()
min_sp = fact_city_readiness['smartphone_penetration'].min()

mean_ip = fact_city_readiness['internet_penetration'].mean()
std_ip = fact_city_readiness['internet_penetration'].std()
max_ip = fact_city_readiness['internet_penetration'].max()
min_ip = fact_city_readiness['internet_penetration'].min()


lower_bound_lr = mean_lr - 3 * std_lr
upper_bound_lr = mean_lr + 3 * std_lr

lower_bound_sp = mean_sp - 3 * std_sp
upper_bound_sp = mean_sp + 3 * std_sp

lower_bound_ip = mean_ip - 3 * std_ip
upper_bound_ip = mean_ip + 3 * std_ip



print(f"literacy_rate: (Lower:{round(lower_bound_lr,2)}, Min: {round(min_lr,2)}, Max: {round(max_lr,2)}, Upper: {round(upper_bound_lr,2)})" )
print(f"smartphone_penetration: (Lower:{round(lower_bound_sp,2)}, Min: {round(min_sp,2)}, Max: {round(max_sp,2)}, Upper: {round(upper_bound_sp,2)})")
print(f"internet_penetration: (Lower:{round(lower_bound_ip,2)}, Min: {round(min_ip,2)}, Max: {round(max_ip,2)}, Upper: {round(upper_bound_ip,2)})")

literacy_rate: (Lower:0.56, Min: 0.66, Max: 0.89, Upper: 0.98)
smartphone_penetration: (Lower:0.37, Min: 0.45, Max: 0.8, Upper: 1.0)
internet_penetration: (Lower:0.04, Min: 0.1, Max: 0.77, Upper: 1.18)


##### All values fall within the upper and lower limits. Hence no outliers detected.

#### Dataset 4: fact_ad_revenue

In [43]:
fact_ad_revenue.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments
0,ED1005,A001,2023-Q2,22613.69,EUR,NaN
1,ED1005,A002,Q1-2019,39366.88,USD,NaN
2,ED1001,A003,Q3-2023,3709860.00,INR,NaN
3,ED1003,A002,Q3-2023,40969.55,USD,NaN
4,ED1007,A003,4th Qtr 2020,51779.40,USD,NaN


In [44]:
fact_ad_revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   edition_id   720 non-null    object 
 1   ad_category  720 non-null    object 
 2   quarter      720 non-null    object 
 3   ad_revenue   720 non-null    float64
 4   currency     720 non-null    object 
 5   comments     38 non-null     object 
dtypes: float64(1), object(5)
memory usage: 33.9+ KB


In [45]:
pd.set_option('display.float_format', '{:,.2f}'.format)
fact_ad_revenue.describe()

,ad_revenue
count,720.00
mean,"2,221,946.34"
std,"1,664,294.36"
min,"11,305.00"
25%,"55,144.49"
50%,"2,271,604.00"
75%,"3,669,044.75"
max,"4,998,136.00"


In [46]:
fact_ad_revenue['quarter'].unique()

array(['2023-Q2', 'Q1-2019', 'Q3-2023', '4th Qtr 2020', '2024-Q2',
       '2019-Q2', 'Q3-2021', '4th Qtr 2023', '2020-Q2', '2022-Q2',
       '2021-Q2', '4th Qtr 2024', 'Q1-2020', 'Q3-2022', 'Q3-2019',
       '4th Qtr 2021', 'Q1-2022', '4th Qtr 2022', 'Q3-2024', 'Q1-2023',
       'Q1-2024', 'Q3-2020', 'Q1-2021', '4th Qtr 2019'], dtype=object)

##### Studying these values carefully, we see the format is different for Q1, Q3 and Q4.
##### We will convert everything to this format YYYY-Qx, i.e. 2019-Q1 and so on..

In [47]:
#defining a function to standarize the values

def clean_quarter(q):
    q = q.strip()
    
    if q == 'Q1-2019': return '2019-Q1'
    elif q == 'Q3-2019': return '2019-Q3'
    elif q == '4th Qtr 2019': return '2019-Q4'
    
    elif q == 'Q1-2020': return '2020-Q1'
    elif q == 'Q3-2020': return '2020-Q3'
    elif q == '4th Qtr 2020': return '2020-Q4'
    
    
    elif q == 'Q1-2021': return '2021-Q1'
    elif q == 'Q3-2021': return '2021-Q3'
    elif q == '4th Qtr 2021': return '2021-Q4'
    
    elif q == 'Q1-2022': return '2022-Q1'
    elif q == 'Q3-2022': return '2022-Q3'
    elif q == '4th Qtr 2022': return '2022-Q4'
    
    elif q == 'Q1-2023': return '2023-Q1'
    elif q == 'Q3-2023': return '2023-Q3'
    elif q == '4th Qtr 2023': return '2023-Q4'
    
    elif q == 'Q1-2024': return '2024-Q1'
    elif q == 'Q3-2024': return '2024-Q3'
    elif q == '4th Qtr 2024': return '2024-Q4'
    
    else:
        return q  # fallback — return as-is


In [48]:
#standardizing values in quarter column
fact_ad_revenue['quarter'] = fact_ad_revenue['quarter'].apply(clean_quarter)
fact_ad_revenue['quarter'].unique()

array(['2023-Q2', '2019-Q1', '2023-Q3', '2020-Q4', '2024-Q2', '2019-Q2',
       '2021-Q3', '2023-Q4', '2020-Q2', '2022-Q2', '2021-Q2', '2024-Q4',
       '2020-Q1', '2022-Q3', '2019-Q3', '2021-Q4', '2022-Q1', '2022-Q4',
       '2024-Q3', '2023-Q1', '2024-Q1', '2020-Q3', '2021-Q1', '2019-Q4'],
      dtype=object)

In [49]:
#splitting quarter column into 2 different columns
fact_ad_revenue['year'] = fact_ad_revenue['quarter'].str[:4]
fact_ad_revenue['quarter_only'] = fact_ad_revenue['quarter'].str[-2:]
fact_ad_revenue.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments,year,quarter_only
0,ED1005,A001,2023-Q2,"22,613.69",EUR,NaN,2023,Q2
1,ED1005,A002,2019-Q1,"39,366.88",USD,NaN,2019,Q1
2,ED1001,A003,2023-Q3,"3,709,860.00",INR,NaN,2023,Q3
3,ED1003,A002,2023-Q3,"40,969.55",USD,NaN,2023,Q3
4,ED1007,A003,2020-Q4,"51,779.40",USD,NaN,2020,Q4


In [50]:
#dropping the old quarter column
fact_ad_revenue.drop(columns=['quarter'], inplace=True)

In [51]:
#Renaming the Quarter Only column with quarter
fact_ad_revenue.rename(columns={'quarter_only': 'quarter'}, inplace=True)
fact_ad_revenue.head()

,edition_id,ad_category,ad_revenue,currency,comments,year,quarter
0,ED1005,A001,"22,613.69",EUR,NaN,2023,Q2
1,ED1005,A002,"39,366.88",USD,NaN,2019,Q1
2,ED1001,A003,"3,709,860.00",INR,NaN,2023,Q3
3,ED1003,A002,"40,969.55",USD,NaN,2023,Q3
4,ED1007,A003,"51,779.40",USD,NaN,2020,Q4


In [52]:
fact_ad_revenue['currency'].unique()

array(['EUR', 'USD', 'INR', 'IN RUPEES'], dtype=object)

In [53]:
fact_ad_revenue['currency'] = fact_ad_revenue['currency'].replace({'IN RUPEES': 'INR'})
fact_ad_revenue['currency'].unique()

array(['EUR', 'USD', 'INR'], dtype=object)

In [54]:
# Normalizing currency to INR
# Conversion rates taken as of 20 September 2025.
# Static conversion rates are taken for project simplicity. Dynamic conversion rates are also possible.

conversion_rates_inr = {
    'USD': 88.09,    
    'EUR': 103.46,    
    'INR': 1         
}

In [55]:
# Converting values to INR

fact_ad_revenue['ad_revenue_inr'] = (
    fact_ad_revenue['ad_revenue'] *
    fact_ad_revenue['currency'].map(conversion_rates_inr)
)

fact_ad_revenue.head()

,edition_id,ad_category,ad_revenue,currency,comments,year,quarter,ad_revenue_inr
0,ED1005,A001,"22,613.69",EUR,NaN,2023,Q2,"2,339,612.37"
1,ED1005,A002,"39,366.88",USD,NaN,2019,Q1,"3,467,828.46"
2,ED1001,A003,"3,709,860.00",INR,NaN,2023,Q3,"3,709,860.00"
3,ED1003,A002,"40,969.55",USD,NaN,2023,Q3,"3,609,007.66"
4,ED1007,A003,"51,779.40",USD,NaN,2020,Q4,"4,561,247.35"


In [56]:
fact_ad_revenue = fact_ad_revenue.drop(columns=['ad_revenue', 'currency'])
fact_ad_revenue.head()

,edition_id,ad_category,comments,year,quarter,ad_revenue_inr
0,ED1005,A001,NaN,2023,Q2,"2,339,612.37"
1,ED1005,A002,NaN,2019,Q1,"3,467,828.46"
2,ED1001,A003,NaN,2023,Q3,"3,709,860.00"
3,ED1003,A002,NaN,2023,Q3,"3,609,007.66"
4,ED1007,A003,NaN,2020,Q4,"4,561,247.35"


In [57]:
#checking for outliers 
mean_ar = fact_ad_revenue['ad_revenue_inr'].mean()
std_ar = fact_ad_revenue['ad_revenue_inr'].std()
max_ar = fact_ad_revenue['ad_revenue_inr'].max()
min_ar = fact_ad_revenue['ad_revenue_inr'].min()


lower_bound_ar = mean_ar - 3 * std_ar
upper_bound_ar = mean_ar + 3 * std_ar


print(f"Ad Revenue: (Lower:{int(lower_bound_ar)}, Min: {int(min_ar)}, Max: {int(max_ar)}, Upper: {int(upper_bound_ar)})")

Ad Revenue: (Lower:-538268, Min: 1005833, Max: 5707415, Upper: 6702870)


##### All values fall within the upper and lower limits. Hence no outliers detected.

#### Dataset 5: dim_city

In [58]:
dim_city.head()

,city_id,city,state,tier
0,C001,lucknow,Uttar Pradesh,Tier 2
1,C002,Delhi,DELHI,Tier 1
2,C003,bhopal,Madhya Pradesh,Tier 2
3,C004,Patna,BIHAR,Tier 2
4,C005,jaipur,Rajasthan,Tier 2


In [59]:
dim_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city_id  10 non-null     object
 1   city     10 non-null     object
 2   state    10 non-null     object
 3   tier     10 non-null     object
dtypes: object(4)
memory usage: 452.0+ bytes


In [60]:
dim_city['city'].unique()

array(['lucknow', 'Delhi', 'bhopal', 'Patna', 'jaipur', 'Mumbai',
       'ranchi', 'kanpur', 'Ahmedabad', 'Varanasi'], dtype=object)

In [61]:
# Changing city names to proper case format
dim_city['city'] = dim_city['city'].str.title()
dim_city['city'].unique()

array(['Lucknow', 'Delhi', 'Bhopal', 'Patna', 'Jaipur', 'Mumbai',
       'Ranchi', 'Kanpur', 'Ahmedabad', 'Varanasi'], dtype=object)

In [62]:
dim_city['state'].unique()

array(['Uttar Pradesh', 'DELHI', 'Madhya Pradesh', 'BIHAR', 'Rajasthan',
       'MAHARASHTRA', 'JHARKHAND', 'UTTAR PRADESH', 'GUJARAT'],
      dtype=object)

In [63]:
# Changing state names to proper case format
dim_city['state'] = dim_city['state'].str.title()
dim_city['state'].unique()

array(['Uttar Pradesh', 'Delhi', 'Madhya Pradesh', 'Bihar', 'Rajasthan',
       'Maharashtra', 'Jharkhand', 'Gujarat'], dtype=object)

In [64]:
dim_city['tier'].unique()

array(['Tier 2', 'Tier 1', 'Tier 3'], dtype=object)

In [65]:
dim_city.head()

,city_id,city,state,tier
0,C001,Lucknow,Uttar Pradesh,Tier 2
1,C002,Delhi,Delhi,Tier 1
2,C003,Bhopal,Madhya Pradesh,Tier 2
3,C004,Patna,Bihar,Tier 2
4,C005,Jaipur,Rajasthan,Tier 2


##### Dataset 6: dim_ad_category

In [66]:
dim_ad_category.head()

,ad_category_id,standard_ad_category,category_group,example_brands
0,A001,Government,Public Sector,"LIC, SBI"
1,A002,FMCG,Commercial Brands,"HUL, Britannia"
2,A003,Real Estate,Private Sector,"DLF, Lodha"
3,A004,Automobile,Commercial Brands,"Tata Motors, Maruti"


In [67]:
dim_ad_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ad_category_id        4 non-null      object
 1   standard_ad_category  4 non-null      object
 2   category_group        4 non-null      object
 3   example_brands        4 non-null      object
dtypes: object(4)
memory usage: 260.0+ bytes


In [68]:
# Splitting example_brands into 2 separate columns.
dim_ad_category[['example_brand_1', 'example_brand_2']] = (
    dim_ad_category['example_brands']
    .str.split(',', n=1, expand=True)
)

In [69]:
# Removing any leading/trailing spaces
dim_ad_category['example_brand_1'] = dim_ad_category['example_brand_1'].str.strip()
dim_ad_category['example_brand_2'] = dim_ad_category['example_brand_2'].str.strip()

# Dropping the old column
dim_ad_category = dim_ad_category.drop(columns=['example_brands'])


In [70]:
dim_ad_category.head()

,ad_category_id,standard_ad_category,category_group,example_brand_1,example_brand_2
0,A001,Government,Public Sector,LIC,SBI
1,A002,FMCG,Commercial Brands,HUL,Britannia
2,A003,Real Estate,Private Sector,DLF,Lodha
3,A004,Automobile,Commercial Brands,Tata Motors,Maruti


In [71]:
# Creating copies of cleaned dataframes

fact_print_sales_cleaned = fact_print_sales.copy()
fact_digital_pilot_cleaned = fact_digital_pilot.copy()
fact_city_readiness_cleaned = fact_city_readiness.copy()
fact_ad_revenue_cleaned = fact_ad_revenue.copy()
dim_city_cleaned = dim_city.copy()
dim_ad_category_cleaned = dim_ad_category.copy()

#### Data Modelling (basics, just adding new dimension tables as required)

In [72]:
# Checking if city id maps to exactly 1 edition id. 
city_edition_counts = fact_print_sales_cleaned.groupby('city_id')['edition_id'].nunique()
city_edition_counts.value_counts()

edition_id
1    10
Name: count, dtype: int64

##### This shows for every city id, we have exactly 1 edition id.
We have 2 apporaches here, either add a new dim_edition table and connect with dim_city as well as fact_ad_revenue, or add edition_id column in dim_city to access cities by edition_id. 
I have used the first approach just to keep things clear.

In [73]:
# adding a new dim_edition table for connection with fact_ad_revenue
dim_edition_cleaned  = fact_print_sales_cleaned[['edition_id', 'city_id']].drop_duplicates()
dim_edition_cleaned.head()

,edition_id,city_id
0,ED1005,C005
2,ED1001,C001
3,ED1003,C003
4,ED1007,C007
9,ED1006,C006


In [74]:
# adding a date dimension table 
start_date = '2019-01-01'
end_date = '2024-12-31'

date_range = pd.date_range(start=start_date, end=end_date, freq='D')

dim_date = pd.DataFrame({'date': date_range})

dim_date.head()

,date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05


In [75]:
#extracting years, months and quarters from dates.

dim_date['year'] = dim_date['date'].dt.year.astype(str)
dim_date['month'] = dim_date['date'].dt.month
dim_date['quarter'] = np.ceil(dim_date['month'] / 3).astype(int)
dim_date['quarter'] = 'Q' + dim_date['quarter'].astype(str)
dim_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192 entries, 0 to 2191
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     2192 non-null   datetime64[ns]
 1   year     2192 non-null   object        
 2   month    2192 non-null   int32         
 3   quarter  2192 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 60.1+ KB


In [76]:
# Creating a table with unique values of years
dim_year = pd.DataFrame({'year': dim_date['year'].unique()})
dim_year = dim_year.sort_values('year').reset_index(drop=True)
dim_year.head()

,year
0,2019
1,2020
2,2021
3,2022
4,2023


In [77]:
# Creating a table with unique values of quarters.
dim_quarter = pd.DataFrame({'quarter': dim_date['quarter'].unique()})
dim_quarter = dim_quarter.sort_values('quarter').reset_index(drop=True)
dim_quarter.head()

,quarter
0,Q1
1,Q2
2,Q3
3,Q4


In [78]:
#Final check before exporting as excel files

print("fact_print_sales_cleaned")
display(fact_print_sales_cleaned.head())

print("\nfact_digital_pilot_cleaned")
display(fact_digital_pilot_cleaned.head())

print("\nfact_city_readiness_cleaned")
display(fact_city_readiness_cleaned.head())

print("\nfact_ad_revenue_cleaned")
display(fact_ad_revenue_cleaned.head())

print("\ndim_city_cleaned")
display(dim_city_cleaned.head())

print("\ndim_ad_category_cleaned")
display(dim_ad_category_cleaned.head())

print("\ndim_ad_category_cleaned")
display(dim_edition_cleaned.head())

print("\ndim_ad_category_cleaned")
display(dim_date.head())

print("\ndim_ad_category_cleaned")
display(dim_year.head())

print("\ndim_ad_category_cleaned")
display(dim_quarter.head())

fact_print_sales_cleaned


,edition_id,city_id,language,state,month,copies_sold,copies_returned,net_circulation
0,ED1005,C005,Hindi,Rajasthan,2023-05-01,404389,13510,390879
1,ED1005,C005,Hindi,Rajasthan,2019-03-01,492943,25024,467919
2,ED1001,C001,Hindi,Uttar Pradesh,2023-07-01,168893,12285,156608
3,ED1003,C003,Hindi,Madhya Pradesh,2023-07-01,216540,10117,206423
4,ED1007,C007,Hindi,Jharkhand,2020-10-01,234563,13048,221515



fact_digital_pilot_cleaned


,platform,launch_month,ad_category_id,dev_cost,marketing_cost,users_reached,downloads_or_accesses,avg_bounce_rate,cumulative_feedback_from_customers,city_id
0,PDF WhatsApp Push,2021-01-01,A001,236570,66060,23509,16319,0.53,"Mixed feedback: some usability concerns, but h...",C001
1,PDF WhatsApp Push,2021-02-01,A001,156865,99122,19472,17017,0.83,"Mixed feedback: some usability concerns, but h...",C002
2,PDF WhatsApp Push,2021-03-01,A001,242728,46087,8471,2891,0.68,"Mixed feedback: some usability concerns, but h...",C003
3,PDF WhatsApp Push,2021-04-01,A001,147695,78868,46796,15640,0.66,"Mixed feedback: some usability concerns, but h...",C004
4,PDF WhatsApp Push,2021-05-01,A001,325906,135644,16805,3231,0.77,The site takes too long to load on average pho...,C005



fact_city_readiness_cleaned


...

,date,year,month,quarter
0,2019-01-01,2019,1,Q1
1,2019-01-02,2019,1,Q1
2,2019-01-03,2019,1,Q1
3,2019-01-04,2019,1,Q1
4,2019-01-05,2019,1,Q1



dim_ad_category_cleaned


,year
0,2019
1,2020
2,2021
3,2022
4,2023



dim_ad_category_cleaned


,quarter
0,Q1
1,Q2
2,Q3
3,Q4


In [79]:
# Saving each cleaned dataframe as a separate Excel file in the same folder

fact_print_sales_cleaned.to_excel("fact_print_sales_cleaned.xlsx", index=False)
fact_digital_pilot_cleaned.to_excel("fact_digital_pilot_cleaned.xlsx", index=False)
fact_city_readiness_cleaned.to_excel("fact_city_readiness_cleaned.xlsx", index=False)
fact_ad_revenue_cleaned.to_excel("fact_ad_revenue_cleaned.xlsx", index=False)
dim_city_cleaned.to_excel("dim_city_cleaned.xlsx", index=False)
dim_ad_category_cleaned.to_excel("dim_ad_category_cleaned.xlsx", index=False)
dim_edition_cleaned.to_excel("dim_edition_cleaned.xlsx", index=False)
dim_date.to_excel("dim_date.xlsx", index=False)
dim_year.to_excel("dim_year.xlsx", index=False)
dim_quarter.to_excel("dim_quarter.xlsx", index=False)

In [81]:
# Exporting files as CSV for MYSQL Adhoc tasks

fact_print_sales_cleaned.to_csv("../SQL/fact_print_sales_cleaned.csv", index=False)
fact_digital_pilot_cleaned.to_csv("../SQL/fact_digital_pilot_cleaned.csv", index=False)
fact_city_readiness_cleaned.to_csv("../SQL/fact_city_readiness_cleaned.csv", index=False)
fact_ad_revenue_cleaned.to_csv("../SQL/fact_ad_revenue_cleaned.csv", index=False)
dim_city_cleaned.to_csv("../SQL/dim_city_cleaned.csv", index=False)
dim_ad_category_cleaned.to_csv("../SQL/dim_ad_category_cleaned.csv", index=False)
dim_edition_cleaned.to_csv("../SQL/dim_edition_cleaned.csv", index=False)
dim_date.to_csv("../SQL/dim_date.csv", index=False)
dim_year.to_csv("../SQL/dim_year.csv", index=False)
dim_quarter.to_csv("../SQL/dim_quarter.csv", index=False)


#### While data visualization is possible using Seaborn and Matplotlib, due to time constraints and reasons mentioned at the top of this notebook, all visualization has been performed in Power BI

### Data Cleaning is done. These exported excel files are used in Power BI for dashboards and exported CSV files are used in MYSQL Workbench for Ad-hoc business requests.